# Just an example.You can alter sample code anywhere.

## mount your google drive

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# You need to modify this part to the directory where your code is located
# %cd "/content/drive/MyDrive/DL_Lab1/"

## Import packages


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import model

In [4]:
#Fix the random seed
np.random.seed(0)

## Load the data and label

In [ ]:
train_load = np.loadtxt('./data/kmnist-train.csv',delimiter=',',dtype="int")
test_load = np.loadtxt('./data/kmnist-test.csv',delimiter=',',dtype="int")

train_data=train_load[:,1:]
train_label=train_load[:,0]
test_data=test_load
print("shape of train_data: {}".format(train_data.shape))
print("shape of train_label: {}".format(train_label.shape))
print("shape of test_data: {}".format(test_data.shape))

## Show the training data

In [6]:
#uncomment if you want to show the training data
#plt.figure(figsize=(20, 20))
#for index in range(10):
#    image = train_data[index+1000].reshape(28,28)
#    plt.subplot(2, 5, index+1)
#    plt.imshow(image)
#plt.show()

In [ ]:
train_image_num = train_data.shape[0]
test_image_num = test_data.shape[0]
train_data = train_data.astype(np.float32)
test_data = test_data.astype(np.float32)

print("train_image_num  is : {}".format(train_image_num))
print("test_image_num   is : {}".format(test_image_num))

## Validation image number

In [8]:
val_image_num=10000

## Convert the training labels to one hot vector

In [ ]:
label_temp = np.zeros((train_image_num, 10), dtype = np.float32)
for i in range(train_image_num):
    label_temp[i][train_label[i]] = 1
train_label_onehot = np.copy(label_temp)
print("One-hot training labels shape:",train_label_onehot.shape)



## Hyperparameters

In [10]:
EPOCH = 30
Batch_size =  10# 10000 should be divisible by batch_size
Learning_rate = 0.005

## Training

In [ ]:
# please make sure you have place layer.py & network.py in 'model' folder
net = model.Network()

train_batch_num = (train_image_num  -  val_image_num  )//Batch_size
val_batch_num = (val_image_num)//Batch_size

for epoch in range(1, EPOCH+1):
    # if epoch > 20:
    #   Learning_rate = 1e-5
    train_hit = 0
    val_hit = 0
    total_train_loss = 0.0
    total_val_loss = 0.0
    for it in range(train_batch_num):
        pred, train_loss = net.forward(train_data[it*Batch_size:(it+1)*Batch_size], train_label_onehot[it*Batch_size:(it+1)*Batch_size])
        pred_index = np.argmax(pred, axis=1)
        train_hit += (pred_index==train_label[it*Batch_size:(it+1)*Batch_size]).sum()
        total_train_loss += train_loss

        net.backward()
        net.update(Learning_rate)

    for titt in range(val_batch_num):
        tit=train_batch_num+titt
        pred, val_loss = net.forward(train_data[tit*Batch_size:(tit+1)*Batch_size], train_label_onehot[tit*Batch_size:(tit+1)*Batch_size])
        pred_index = np.argmax(pred, axis=1)
        val_hit += (pred_index==train_label[tit*Batch_size:(tit+1)*Batch_size]).sum()
        total_val_loss += val_loss

    print('Task1  | Epoch:%3d'%epoch, ' |Train Loss:%8.4f'%(total_train_loss/train_batch_num), ' |Train Acc:%3.4f'%(train_hit/(train_image_num-val_image_num)*100.0)
          , ' |Val Loss:%8.4f'%(total_val_loss/val_batch_num), ' |Val Acc:%3.4f'%(val_hit/val_image_num*100.0))

## Dump for evaluation (upload your DL-test-predict.csv to kaggle )

In [ ]:
test_pred_list = []
total_test = 0
for tit in range(test_image_num//Batch_size):
    pred, _ = net.forward(test_data[tit*Batch_size:(tit+1)*Batch_size], train_label_onehot[tit*Batch_size:(tit+1)*Batch_size])
    pred_index = np.argmax(pred, axis=1)
    test_pred_list += pred_index.tolist()
    total_test += Batch_size

print('Please make sure that total test images = 10000')
print(f'Total test images: {total_test} ')

print('Dump file...')
df = pd.DataFrame(test_pred_list, columns=["Category"])
df.to_csv('DL-test-predict.csv', index=True, index_label="Id")